In [ ]:
def calc_spectrum_euan(data, sample_interval, method='none', prnt=False):
    
    """
    CALCULATES THE SPECTRUM FOR A GIVEN TIMESERIES.
    
    Inputs
    ------
    data: list
        List containing arrays of data
    sample_interval: float
        Sample interval between points in chosen units
    method: str
        Determines how the data is processed before transforming it. 'none' for raw data,
        'detrend' for detrended data with zero mean, 'hanning' for detrending and hanning window
    prnt: boolean
        True to print the number of chunks of data, False otherwise
        
    Outputs
    -------
    absX: array
        Normalised spectrum associated with each frequency
    f: array
        Array of frequency values
    """
    
    # For this question, the chunk size we want is 60 days (which is 6*24*60 10 minute periods).
    # The chunk overlap is 50%
    chunk_size = 6*24*60
    chunk_overlap = 0.5

    # Figure out how many chunks are in each year of data (this works for one year or multiple years)
    no_chunks = np.zeros(len(data))
    for i in range(len(data)):
        length = len(data[i])
        no_chunks[i] = int(np.floor(length/(chunk_size*chunk_overlap))-1)
    
    # The total number of chunks is the sum of the chunks associated with each year of data
    total_chunks = int(np.sum(no_chunks))
    
    # Create an empty array of the right size to Fourier transform
    fourier_data = np.zeros((chunk_size, total_chunks))
    count = 0
    
    # Print the number of segments in the dataset
    if prnt == True:
        print('The total number of segments in the data file is', total_chunks)
    
    # Run a loop over the number of different years of data (the reason I do this is because there are gaps at the end of each
    # year of data and I don't want segments to run between the two years)
    for i in range(len(data)):
        
        # Run an inner loop over the number of chunks in each year of data
        for j in range(int(no_chunks[i])):
            # Only segment the data is method is 'none'
            if  method == 'none':
                fourier_data[:, count:count+1] = data[i][int((j*0.5)*chunk_size):int((j*0.5+1)*chunk_size)]
                count += 1
                
            # Detrend (and remove mean) if method is 'detrend'
            elif method == 'detrend':
                chunked_data = data[i][int((j*0.5)*chunk_size):int((j*0.5+1)*chunk_size)]
                fourier_data[:, count:count+1] = signal.detrend(chunked_data, axis=0)
                count += 1
                
            # Detrend and multiply by a normalised Hanning window if method is 'hanning'
            elif method == 'hanning':
                w = np.hanning(chunk_size)
                w_normalised = (w**2 / np.mean(w**2))**0.5
                chunked_data = data[i][int((j*0.5)*chunk_size):int((j*0.5+1)*chunk_size)][:,0]* w_normalised    
                fourier_data[:, count:count+1] = np.expand_dims(signal.detrend(chunked_data, axis=0), axis=1)
                count += 1
    
    # N is the number of points in each chunk
    # M is the number of chunks
    N = np.shape(fourier_data)[0]
    M = np.shape(fourier_data)[1]
    
    dt = sample_interval    # The time spacing is given by the sample interval
    T = dt*N                # Total length is 60 days
    df = 1/T                # Frequency spacing is 1/T
    fn = 1/(2*dt)           #The Niquist frequency is 72 per day for a sample interval of 10 minutes

    # Find the frequency vector ranging from 0 to the Niquist frequency
    f = np.arange(0, fn, df)
    
    X = np.fft.fft(fourier_data, axis=0)
    
    absX = abs(X[0:int(N/2),:])**2

    absX = absX*2 # We threw out half of the spectrum; so correct for the lost variance
    absX = absX/N**2 # First correct for the normalization
    absX = absX/df # This is then the definition of the spectrum
    
    return f, absX